#### Importamos las librerias necesarias para realizar nuetro EDA:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
from sklearn.metrics.pairwise import cosine_similarity

#### Cargamos el datasets movies_EDA_relevantes.csv, resultado del EDA en un DataFrame, revisamos su estructura y su informacion:

In [ ]:
# Cargar los datos en un DataFrame
df = pd.read_csv('movies_EDA_relevantes.csv', low_memory=False)

In [ ]:
df.head()

,genres,id,original_language,overview,popularity,production_companies,production_countries,runtime,spoken_languages,title,vote_average,release_year
0,"Animation, Comedy, Family",862,en,"Led by Woody, Andy's toys live happily in his ...",3.133185,Pixar Animation Studios,United States of America,81.0,English,Toy Story,7.7,7.5989
1,"Adventure, Fantasy, Family",8844,en,When siblings Judy and Peter discover an encha...,2.891235,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,104.0,"English, Français",Jumanji,6.9,7.5989
2,"Romance, Comedy",15602,en,A family wedding reignites the ancient feud be...,2.542617,"Warner Bros., Lancaster Gate",United States of America,101.0,English,Grumpier Old Men,6.5,7.5989
3,"Comedy, Drama, Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",1.580935,Twentieth Century Fox Film Corporation,United States of America,127.0,English,Waiting to Exhale,6.1,7.5989
4,Comedy,11862,en,Just when George Banks has recovered from his ...,2.239381,"Sandollar Productions, Touchstone Pictures",United States of America,106.0,English,Father of the Bride Part II,5.7,7.5989


In [ ]:
df.info()

#### Planteamos un sistema de recomendación de películas basado en contenido. Reducimos la muestra para asegurar su funcionamiento en el deploy.

* Seleccionamos columnas relevantes para el modelo.
* Codificación one-hot para la columna 'genres'.
* Creación de una matriz de características dispersa para 'overview' utilizando TF-IDF
* Concatenamos las matrices de características y las características categóricas
* Cálculo de la similitud del coseno entre las características, el resultado es la matriz similarity_matrix que representa la similitud entre todas las películas basada en las características seleccionadas.
* Creación del diccionario que mapea el título de la película a su índice en el DataFrame.

In [ ]:
# Reducción de la muestra de datos
sample_frac = 0.4  # Ejemplo: utilizar el 40% de los datos
df_sample = df.sample(frac=sample_frac, random_state=42)

# Seleccionar las columnas relevantes
relevant_cols = ['title', 'genres', 'original_language', 'overview']

# Crear un nuevo DataFrame con las columnas relevantes
df_relevant = df_sample[relevant_cols].copy()

# Tratar los valores faltantes
df_relevant.fillna({'genres': 'Unknown', 'original_language': 'Unknown', 'overview': ''}, inplace=True)

# Codificación one-hot para la columna 'genres'
one_hot_encoder = OneHotEncoder(sparse_output=False)
genres_encoded = one_hot_encoder.fit_transform(df_relevant[['genres']])

# Crear una matriz de características dispersa para el resumen (overview) utilizando TF-IDF
vectorizer = TfidfVectorizer()
overview_features = vectorizer.fit_transform(df_relevant['overview'])

# Concatenar las matrices de características y las características categóricas
features_matrix = hstack([overview_features, genres_encoded])

# Calcular la similitud del coseno entre las características
similarity_matrix = cosine_similarity(features_matrix)

# Crear un diccionario que mapee el título de la película a su índice en el DataFrame
title_to_index = {title: index for index, title in enumerate(df_sample['title'])}


In [ ]:
# Generar recomendaciones basadas en el contenido
def recomendacion(titulo: str, top_n=5):
    # Obtener el índice de la película objetivo
    movie_index = title_to_index[titulo]

    # Obtener la fila de similitud correspondiente a la película objetivo
    similarity_scores = similarity_matrix[movie_index]

    # Obtener los índices de las películas más similares
    top_indices = similarity_scores.argsort()[::-1][1:top_n+1]

    # Obtener los títulos de las películas recomendadas
    recommended_movies = df_sample.iloc[top_indices]['title'].values.tolist()

    return {'Lista recomendada': recommended_movies}

In [ ]:
df_sample['title'].tail(20)

17847    I Am My Films: A Portrait of Werner Herzog
37343                         We Have Only One Life
7690                             The Stepford Wives
32216                                   La Habanera
13594            Wild Things: Diamonds in the Rough
29232                                Monkey Kingdom
35373                            Proof of the Devil
42371                              The Glass Castle
2173                                    Strangeland
41392                              City of Dead Men
18190                                 Metrobranding
6257                               13 Rue Madeleine
10070                        The Captain's Paradise
41451                        The Over-the-Hill Gang
11793                       Godzilla vs. Destoroyah
9250                                  My Little Eye
12022                                  Mr. Woodcock
43160                                    Pet Fooled
2819                                         Dr. No
4105        

In [ ]:
# Generar recomendaciones para una película específica
titulo = 'Toy Story 2'
recomendacion(titulo)

{'Lista recomendada': ['Toy Story',
  'Hop',
  'Scooby-Doo! Camp Scare',
  "Bébé's Kids",
  'Lorenzo']}

#### Teniendo en cuanta que nuestro modelo solo funciona con tres columnas, vamos a crear un datasets con esas columnas y con el % de los datos asignados, asi evitamos el uso innecesario de recursos:

In [ ]:
# Reducción de la muestra de datos
sample_frac = 0.4  # Se utiliza el 40% de los datos del dataset.
df_sample = df.sample(frac=sample_frac, random_state=42)

# Seleccionar las columnas relevantes
relevant_cols = ['title', 'genres', 'overview']

# Crear un nuevo DataFrame con las columnas relevantes
df_relevant = df_sample[relevant_cols]

# Tratar los valores faltantes, si los hubiera
df_relevant.fillna({'genres': 'Unknown', 'overview': 'Unknown'}, inplace=True)

#Exportamos a csv
df_relevant.to_csv('movies_relevantes_ML.csv', index=False)

#### Cargamos el nuevo dataset y vemos como se reducen algunas lines de codigo:

In [ ]:
df2 = pd.read_csv('movies_relevantes_ML.csv', low_memory=False)
df2.info()

In [ ]:
# Tratar los valores faltantes, si los hubiera
df2.fillna({'genres': 'Unknown', 'overview': ''}, inplace=True)

# Codificación one-hot para la columna 'genres'
one_hot_encoder = OneHotEncoder(sparse_output=False)
genres_encoded = one_hot_encoder.fit_transform(df2[['genres']])

# Crear una matriz de características dispersa para el resumen (overview) utilizando TF-IDF
vectorizer = TfidfVectorizer()
overview_features = vectorizer.fit_transform(df2['overview'])

# Concatenar las matrices de características y las características categóricas
features_matrix = hstack([overview_features, genres_encoded])

# Calcular la similitud del coseno entre las características
similarity_matrix = cosine_similarity(features_matrix)

# Crear un diccionario que mapee el título de la película a su índice en el DataFrame
title_to_index = {title: index for index, title in enumerate(df2['title'])}

In [ ]:
def recomendacion(titulo: str, top_n=5):
    # Obtener el índice de la película objetivo
    movie_index = title_to_index[titulo]

    # Obtener la fila de similitud correspondiente a la película objetivo
    similarity_scores = similarity_matrix[movie_index]

    # Obtener los índices de las películas más similares
    top_indices = similarity_scores.argsort()[::-1][1:top_n+1]

    # Obtener los títulos de las películas recomendadas
    recommended_movies = df2.iloc[top_indices]['title'].values.tolist()

    return {'Lista recomendada': recommended_movies}

In [ ]:
titulo = 'Toy Story'
recomendacion(titulo)

{'Lista recomendada': ['Toy Story 2',
  'A Boy Named Charlie Brown',
  'Creature Comforts',
  'Barnyard',
  'One Froggy Evening']}